#  

# Mini Project 4

By the deadline (**26.01.2026**), please submit the provided Jupyter notebook with all/some required tasks completed and clearly solved. Make sure your code is neat, well-commented, and that all outputs are visible (run all cells before saving). Notebooks with missing tasks or unexecuted cells may receive fewer points. After you submit, you won’t be able to make changes, so double-check your work and be sure to start from the provided template

## Submission rules
As already discussed in class, we will stick to the following rules.
- Use the templates and name your files `NAME_SURNAME.ipynb` (If you have more than one name, just concatenate them). We will compare what you present with that file. 
- Code either not written in Python or not using PyTorch receives a grade of 0. Of course, you can use auxiliary packages when needed (`matplotlib`, `numpy`, `networkx`...), but for the learning part, you must use PyTorch.
-  If plagiarism is suspected, TAs and I will thoroughly investigate the situation, and we will summon the student for a face-to-face clarification regarding certain answers they provided. In case of plagiarism, a score reduction will be applied to all the people involved, depending on their level of involvement.
-  If extensive usage of AI tools is detected, we will summon the student for a face-to-face clarification regarding certain answers they provided. If the answers are not adequately supported with in-person answers, we will proceed to apply a penalty to the evaluation, ranging from 10% to 100%.

## Traveling Salesperson Problem (TSP) – Heuristic with Transformers

### Introduction

The Traveling Salesperson Problem (TSP) is a classical combinatorial optimization problem: given a set of cities and the distances between them, the goal is to find the shortest possible tour that visits each city exactly once and returns to the starting point. See this picture for reference. 

![TSP](https://optimization.cbe.cornell.edu/images/e/ea/48StatesTSP.png)

TSP is NP-hard, meaning that finding the optimal solution becomes computationally intractable as the number of cities grows. Therefore, in practice, we often rely on heuristic or approximate algorithms to find good solutions in a reasonable time.

In this assignment, your task is to implement a heuristic TSP solver for Euclidean TSP using a Transformer-based model. 

Euclidean TSP is a TSP where all the cities are assumed to be in the Euclidean 2D space and the distance is computed acording to the standard Euclidean norm.

The Transformer architecture, widely used in natural language processing, can also model sequences of nodes in a graph. By training a Transformer on TSP instances, you can generate tours in an autoregressive manner selecting one city at a time based on previously chosen cities.

This assignment will help you to understand how sequence models like Transformers can be applied beyond text, to combinatorial problems and practice autoregressive generation and masking techniques for sequence prediction.

Most of the code is provided. This **does not mean that you are allowed to use it blindly**: make sure to understand what is happening.

In [ ]:
import networkx as nx
import torch
import torch.nn as nn
import math
from networkx.algorithms.approximation import greedy_tsp
import numpy as np
import pickle
import torch.optim as optim
from tqdm import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Task 1 (0 pts)

Learn how to use the [NetworkX](https://networkx.org/documentation/stable/tutorial.html) package. The section on Graphs should be sufficient for this assignment. You can skip the sections on directed graphs and beyond.


### Utility Functions

Below are some utility functions. Detailed explanations are provided in each function’s description. At a high level, these functions do the following:

* `random_tour`: Takes a `networkx.Graph` as input and returns a random tour. This can serve as a simple heuristic, but its performance is generally low, as expected.
* `tour_length`: Takes a `networkx.Graph` and a tour (encoded as a `list` that starts and ends at the same node) and returns the total length of the tour.
* `gap`: Measures the quality of a tour by computing

$$ \dfrac{\text{Estimated} - \text{Optimal}}{\text{Optimal}} $$

As you might expect, lower values are better, with a gap of zero indicating that the tour is exactly optimal.


In [3]:
def tour_length(G: networkx.Graph, tour: List[int]) -> float:
    """
    Generate a random TSP tour for a given graph.

    Parameters
    ----------
    G : networkx.Graph
        A graph representing the TSP instance. Nodes should be numbered from 0 to n-1.

    Returns
    -------
    tour : list of int
        A list of node indices representing a complete tour starting and ending at node 0.
        Each node appears exactly once in the tour (except the start/end node).
    """
    n = G.number_of_nodes()
    tour = [0]
    for i in range(1, n):
        next_node = np.random.choice([j for j in range(n) if j not in tour])
        tour.append(next_node)
    tour.append(0)
    return tour




def gap(estimated : float, optimal : float) -> float:
    """
    Compute the relative optimality gap of a TSP tour.

    Parameters
    ----------
    estimated : float
        The total cost of the computed tour.
    optimal : float
        The cost of the known optimal tour.

    Returns
    -------
    gap : float
        The relative gap between the estimated and optimal cost,
        computed as (estimated - optimal) / optimal.
    """
    return (estimated - optimal) / optimal

### Task 2 (10 pts)

The dataset has already been prepared for your convenience and is split into training, validation, and test sets. Load it using [pickle](https://docs.python.org/3/library/pickle.html).

For this task, focus on the case **$n = 20$**, i.e., solving a TSP with 20 cities. Inspect each item in the dataset: what type is it? You should see that it is a `tuple` consisting of a `networkx` graph and a list representing the optimal tour (our ground truth).

Answer the following questions carefully:

* What does the node attribute `pos` represent?
* What does the edge attribute `weight` represent?
* What does the boolean edge attribute `tour` represent?

Do not proceed until you fully understand each of these components.


In [ ]:
# TODO

### Taks 3 (4 pts)

In this task, you will implement a custom PyTorch Dataset to handle TSP instances. Your dataset should take as input a list of tuples, where each tuple contains:

1. A `networkx` graph representing the cities and their distances.
2. A list representing the optimal tour (ground truth).

You should implement a class `TSPDataset` that returns, for a given index `idx`, a tuple `(X, opt_tour)` where:

  * `X` is a tensor of shape `[n_nodes, 2]` containing the coordinates of each node (from the node attribute `pos`).
  * `opt_tour` is a tensor containing the optimal tour as a sequence of node indices.

Then, create a dataset object for both training, validation and test sets.

In [4]:
class TSPDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        pass

    def __len__(self):
        pass

    def __getitem__(self, idx):
        pass

### Task 4 (3 pts)

Create the `Dataloader` objects for training and validation. Probably you just need training and validation loader.

In [ ]:
# TODO

### Task 5 (5 pts)

Implement Positional encoding for sequence models. Positional encoding allows a Transformer to incorporate information about the order of elements in a sequence, since the model itself is permutation-invariant.

You are asked to implement a function/a class that takes as input:

  * `seq_len`: the length of the sequence (number of positions).
  * `embed_dim`: the dimensionality of the embedding vectors.

And returns a tensor of shape `[seq_len, embed_dim]`, where each row corresponds to the positional encoding vector for a position in the sequence.

In [5]:
# Option 1
def positional_encoding(seq_len, embed_dim):
    pass

# Option 2
class PositionalEncoding(nn.Module):
    def __init__(self,max_seq_len, embed_dim):
        pass

    def forward(self, x):
        pass

### Task 6 (30 pts)

In this task, you are asked to implement a Transformer-like architecture. You have two options:

1. Implement the architecture provided in the pdf and sketched with guidance in the next cell. This option is fully guided, however, following it exactly will give you full points.

2. Implement your own Transformer-based architecture. You are free to design it as you wish, as long as it is Transformer-based. No extra points will be awarded for choosing this option, but full points can still be earned if it works correctly.

Choose the approach that best suits your preference and experience.

In [ ]:
class TSPTransformer(nn.Module):
    def __init__(self,
        n: int,
        d_model: int,
        nhead_encoder: int,
        nhead_decoder: int,
        n_encoder_layers: int,
        n_decoder_layers: int,
        dropout: float = 0.1,
    ):
        super().__init__()

        # Things to store, you need them also in the forward
        self.d_model = d_model

        ######################################
        # Feature expansion
        #####################################
        # Linear layer

        # Embedding

        ######################################
        # Encoder
        #####################################
        # Encoder layer

        # Encoder

        ######################################
        # Output linear layer
        #####################################
        

        ######################################
        # Decoder
        #####################################
        
        # Positional Embedding
        
        # Decoder layer

        # Decoder
        
        
        ######################################
        # Output linear layer
        #####################################
        
        # Don't forget the "Masked" attention!

        # Initialize the weight as commonly done for the LLMs
        self.apply(self.init_weights)

    def forward(self, X, src):
        pass
        

    def create_mask(self, seq_len):
        pass
        
    def init_weights(self, m):
        pass

### Task 7 (35 pts)

Train your model. Plot training and validation loss, clearly showing no overfitting.

In [ ]:
# TODO

### Task 8 (3 pts)

Finally, evaluate your model as if it were a heuristic for solving the TSP. Compare its performance with other simple baselines, such as:

* A random tour sampled uniformly at random
* The greedy heuristic, where you start from a given city and repeatedly move to the closest unvisited one. Its theoretical performance guarantees are quite weak, but in practice it can sometimes work surprisingly well.

The code needed to run these baselines is already provided in the next cells. There is just something you may want to check in the second cell after this one.
Before using it, make sure you fully understand what each part of the provided code is doing.

At the very end, you should see something like this, with our trained model beating all the baselines:

```
Greedy TSP average gap: 0.171123
Random TSP average gap: 1.745515
Our TSP average gap: 0.05374
```



In [ ]:
def transformer_tsp(G, model, start_node = 0, DEVICE="cpu"):
    """
    Evaluate a trained Transformer-based TSP heuristic on a given graph G.

    This function performs greedy autoregressive decoding: starting from node 0,
    it repeatedly queries the model to predict the next city to visit, always
    selecting the highest-scoring unvisited node. The predicted tour is finally
    closed by returning to the starting point.

    Parameters
    -----------
        G (networkx.Graph):
            A TSP instance where each node has a 'pos' attribute containing its
            2D coordinates.
        model (torch.nn.Module):
            A trained Transformer-like neural network that takes:
                - x: a tensor of node coordinates  
                - y: the partial tour so far       
              and outputs logits for the next node at each decoding step.
        start_node (int):
            The starting node; default is zero.
        DEVICE (str, optional):
            Device to run the model on (“cpu” or “cuda” (<-- GPU)). Default is "cpu".

    Returns:
    --------
        float:
            The length of the predicted tour, computed using `tour_length`.
    """

    # Set the model in evaluation mode (important for layers like dropout)
    model.eval()

    # Number of nodes
    n = G.number_of_nodes()

    # Extract node coordinates into a tensor of shape [n, 2]
    attr = nx.get_node_attributes(G, "pos")
    x = []
    for i in range(n):
        x.append(torch.tensor(attr[i], dtype=torch.float32))
    x = torch.stack(x)

    # Initial tour starts at node 0
    tour = [start_node]
    y = torch.tensor(tour, dtype=torch.long)

    # Move tensors to the desired device and add batch dimension
    x = x.to(DEVICE).unsqueeze(0)
    y = y.to(DEVICE).unsqueeze(0)

    # First forward pass
    out = model(x, y)

    # Autoregressive greedy decoding
    while len(tour) < n:
        # Get nodes sorted by descending score
        _, idx = torch.topk(out, n, dim=2)

        # Pick the highest-scoring unvisited node
        for i in range(n):
            if idx[0, -1, i] not in tour:
                tour.append(idx[0, -1, i])
                break

        # Update decoder input and run model again
        y = torch.tensor(tour, dtype=torch.long).to(DEVICE).unsqueeze(0)
        out = model(x, y)

    # Close the tour by returning to the starting node
    tour = [int(i) for i in tour] + [start_node]

    # Compute and return its length
    return tour_length(G, tour)

In [ ]:
"""
TODO
1. Load / Use the model you want to use (I guess, your best/last model)
2. Have you loaded the training data?
"""

In [ ]:
model = None
X_test = None

optimal_tours_values = []
greedy_tsp_values = []
random_tsp_values = []
our_heuristic_values = []

for idx, (G, y_opt) in tqdm(enumerate(X_test), total=len(X_test)):
    # OPT
    optimal_tours_values.append(tour_length(G, y_opt))

    # Greedy TSP
    tour_greedy = greedy_tsp(G)
    greedy_tsp_values.append(tour_length(G, tour_greedy))

    # Random TSP
    tour_random = random_tour(G)
    random_tsp_values.append(tour_length(G, tour_random))

    # Our Heuristic
    our_heuristic_values.append(transformer_tsp(G, model, DEVICE=DEVICE))

In [ ]:
# Greedy TSP gap
gaps_greedy = []
for i in range(len(X_test)):
    gap_greedy = gap(greedy_tsp_values[i], optimal_tours_values[i])
    gaps_greedy.append(gap_greedy)
print("Greedy TSP average gap:", np.mean(gaps_greedy), flush=True)

# Random TSP gap
gaps_random = []
for i in range(len(X_test)):
    gap_random = gap(random_tsp_values[i], optimal_tours_values[i])
    gaps_random.append(gap_random)
print("Random TSP average gap:", np.mean(gaps_random), flush=True)

# Our Heuristic
gaps_our = []
for i in range(len(X_test)):
    gap_our = gap(our_heuristic_values[i], optimal_tours_values[i])
    gaps_our.append(gap_random)
print("Our TSP average gap:", np.mean(gaps_our), flush=True)

## Questions

1. Can you use this model for a TSP instance with 50 nodes? How can you improve it?
2. What is attention and why is it the core mechanism of Transformers?
3. How does multi-head attention differ from single-head attention? Why is it useful?
4. What is the purpose of positional encodings in a Transformer? Why are they needed?
5. Describe the role of the feed-forward network inside each Transformer layer.
6. Why do Transformers use Layer Normalization instead of Batch Normalization?
7.  What are Q, K, and V? Explain their dimensions intuitively.
8. Why do we scale the dot product by √dₖ in scaled dot-product attention?
9. What is the intuition behind causal masking?
11. What is the difference between training-time input and inference-time input in autoregressive models? What is teacher forcing? Why is it used?
12. What is tokenization and why does it matter for LLM performance?
13. What is the difference between pretraining and finetuning?
14. Explain greedy decoding vs. sampling vs. beam search.